In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import tensorflow as tf
import os
import json
import numpy as np
from PIL import Image
import cv2
import sys
import matplotlib.pyplot as plt
from pathlib import Path
import datetime

sys.path.append('..')
import food_identification.models.model_helpers as model_helpers
import data.data_helpers as data_helpers
from tensorflow.keras.applications.inception_v3 import preprocess_input as preprocess_inception_v3
from tensorflow.keras.models import Sequential, save_model, load_model

print(sys.executable)
tf.__version__

In [ ]:
CLASSES = ['almond', 'apple', 'apricot', 'avocado', 'banana', 'beef', 'blackberry', 'blueberry', 'broccoli', 'cabbage', 
           'carrot', 'cauliflower', 'celery', 'cheese', 'cherry', 'chicken_breast', 'chocolate', 'corn', 'cucumber',
           'egg', 'eggplant', 'fig', 'grapefruit', 'grapes', 'grated_cheese', 'kiwi', 'lemon', 'lettuce', 'lime',
           'mango', 'melon', 'mushroom', 'olive', 'onion', 'orange', 'other', 'paprika', 'passionfruit','peach',
           'pear','pineapple', 'plum', 'pomegranate', 'pork', 'radish', 'raspberry', 'salami', 'scallion',
           'strawberry', 'tomato', 'watermelon', 'whole_chicken', 'zucchini']

IND2CLASS = {0: 'almond', 1: 'apple', 2: 'apricot', 3: 'avocado', 4: 'banana', 5: 'beef', 6: 'blackberry', 7: 'blueberry',
             8: 'broccoli', 9: 'cabbage', 10: 'carrot', 11: 'cauliflower', 12: 'celery', 13: 'cheese', 14: 'cherry',
             15: 'chicken_breast', 16: 'chocolate', 17: 'corn', 18: 'cucumber', 19: 'egg', 20: 'eggplant', 21: 'fig',
             22: 'grapefruit', 23: 'grapes', 24: 'grated_cheese', 25: 'kiwi', 26: 'lemon', 27: 'lettuce', 28: 'lime',
             29: 'mango', 30: 'melon', 31: 'mushroom', 32: 'olive', 33: 'onion', 34: 'orange', 35: 'other', 36: 'paprika',
             37: 'passionfruit', 38: 'peach', 39: 'pear', 40: 'pineapple', 41: 'plum', 42: 'pomegranate', 43: 'pork',
             44: 'radish', 45: 'raspberry', 46: 'salami', 47: 'scallion', 48: 'strawberry', 49: 'tomato', 50: 'watermelon',
             51: 'whole_chicken', 52: 'zucchini'}


CLASS2IND = {v:k for (k,v) in IND2CLASS.items()}

FINAL_MODEL_PATH = '../food_identification/models/final_model/'
EXPORT_DIR = 'output'
EXPORT_MODEL = 'exported_model'

#### Load final (trained) model

In [ ]:
loaded_model = load_model(
    FINAL_MODEL_PATH,
    custom_objects=None,
    compile=True
)

#### Deploy model

- Add Input layer for processing JSON inputs
- Add preprocessing layer as a part of the deployed model
- Take probabilities and class predicitions as outputs

In [ ]:
# helper functions for deployment 

def prepare_prediction_image(image_str_tensor):
    """Prepare an image tensor for prediction.
    Takes a string tensor containing a binary jpeg image and returns
    a tensor object of the image with dtype float32.

    Parameters:
        image_str_tensor: a tensor containing a binary jpeg image as a string
    Returns:
        image: A tensor representing an image.
    """
    image_str_tensor = tf.cast(image_str_tensor, tf.string)
    image = tf.image.decode_jpeg(image_str_tensor, channels=3)
    image = tf.cast(image, dtype=tf.uint8)
    return image

def prepare_prediction_image_batch(image_str_tensor):
    """Prepare a batch of images for prediction."""
    return tf.map_fn(prepare_prediction_image, image_str_tensor,
                     dtype=tf.uint8)

In [ ]:
prediction_input = tf.keras.Input(
    dtype=tf.string, name='bytes', shape=())

prediction_output = tf.keras.layers.Lambda(
    prepare_prediction_image_batch)(prediction_input)

prediction_output = tf.cast(prediction_output, dtype=tf.float32)
prediction_output = preprocess_inception_v3(prediction_output)

prediction_output = loaded_model(prediction_output)

prediction_output = tf.keras.layers.Lambda(
        lambda x: x, name='PROBABILITIES')(prediction_output)

prediction_class = tf.keras.layers.Lambda(
        lambda x: tf.argmax(x, 1), name='CLASSES')(prediction_output)

ml_model = tf.keras.models.Model(prediction_input, outputs=[prediction_class, prediction_output])

export_dir = 'output'
model_dir = 'exported_model'
model_path = Path(export_dir) / model_dir
if model_path.exists():
    timestamp = datetime.datetime.now().strftime("-%Y-%m-%d-%H-%M-%S")
    model_path = Path(str(model_path) + timestamp)

# save model
tf.saved_model.save(ml_model, str(model_path))

#### Test deployment

- Save two sample JSON files, one single window and a full image stack

- Test deployment locally using:

`gcloud ai-platform local predict --model-dir <EXPORT_DIR>/<EXPORT_MODEL> \
  --json-instances check_deployed_model/<JSON_FILE> \
  --framework tensorflow`
  
- Deploy model on Google AI platform using the guide (https://cloud.google.com/ai-platform/prediction/docs/deploying-models)

- Test remote deployment.


In [ ]:
# load example image
file = '../../deep-food_old/data/validation_real/example15.jpg'

img = np.array(Image.open(file))
img = data_helpers.resize_image_to_1024(img)
plt.imshow(img)
plt.xticks([])
plt.yticks([])

# convert to strided stack
kernel_size = 224
new_img, x0, y0 = model_helpers.imageRGB_as_strided(img, kernel_size=kernel_size, stride=112)

stack = new_img.reshape((-1, kernel_size, kernel_size, 3))
x0 = x0.flatten()
y0 = y0.flatten()

In [ ]:
# generate sample instance
im_test, encoded_string = data_helpers.convert_numpy_image_to_json_instance(stack[15])
with open('check_deployed_model/test_1_window.json', 'w') as fp:
    json.dump({'bytes': {'b64': encoded_string}}, fp)

im_test

In [ ]:
# generate sample stack
instances = data_helpers.encode_stack_as_JSON(stack)

with open('check_deployed_model/test_stack.json', 'w') as f:
    for ins in instances:
        f.write(json.dumps(ins)+'\n')


#### Test core functionality

In [ ]:
# get predictions from deployed model on Google AI platform

# set path to key credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "my-ml6-challenge-2462e6522a17.json"

# specifiy model and version
project = 'my-ml6-challenge'
model = 'food_classifier'
version = 'v0_1'

responses = model_helpers.ai_platform_predict_json(project, model, instances, version=version)

#### Test main function

In [ ]:
# set path to key credentials
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "my-ml6-challenge-2462e6522a17.json"

model_info = {'project': 'my-ml6-challenge',
              'model': 'food_classifier', 
              'version': 'v0_1'}

pred_labels, probabilities, x0, y0, windowsize = \
    model_helpers.object_detection_sliding_window(model=model_info, 
                                                  input_img=img, 
                                                  preprocess_function=preprocess_inception_v3, 
                                                  kernel_size=224, 
                                                  ind2class=IND2CLASS, 
                                                  scaling_factors=[1.5], 
                                                  sliding_strides=[64], 
                                                  thr=0.95, 
                                                  overlap_thr=0.2)


In [ ]:
fig = model_helpers.visualize_predictions(img, 
                                          pred_labels, 
                                          probabilities, 
                                          x0, 
                                          y0,
                                          windowsize)